# Train GMR

In [1]:
using AutoDrivers
using AutoDrivers.GaussianMixtureRegressionDrivers

const EXTRACT_CORE = true
const EXTRACT_TEMPORAL = false
const EXTRACT_WELL_BEHAVED = true
const EXTRACT_NEIGHBOR_FEATURES = false
const EXTRACT_CARLIDAR_RANGERATE = true
const CARLIDAR_NBEAMS = 20
const ROADLIDAR_NBEAMS = 0
const ROADLIDAR_NLANES = 2

2

In [2]:
include("load_train_test_split.jl")

include(joinpath(ROOT_FILEPATH, "pull_traces", "multifeatureset.jl"))
extractor = MultiFeatureExtractor(EXTRACT_CORE, EXTRACT_TEMPORAL, 
                                    EXTRACT_WELL_BEHAVED, EXTRACT_NEIGHBOR_FEATURES, 
                                    EXTRACT_CARLIDAR_RANGERATE, CARLIDAR_NBEAMS,
                                    ROADLIDAR_NBEAMS, ROADLIDAR_NLANES)

assignment = load_assignment()
evaldata = load_evaldata()

nothing

In [3]:
foldset = foldset_match(assignment, FOLD_TRAIN)
context = IntegratedContinuous(NGSIM_TIMESTEP, 1)
params = GMRTrainParams(AccelTurnrate, extractor, context,
    max_n_train_examples=1000000,
    max_n_indicators=10,
    n_components=2,
)

YX_means_stdevs = GaussianMixtureRegressionDrivers._pull_YX_means_stdevs(evaldata, params, foldset)
nothing

In [ ]:
model = fit(evaldata, params, foldset, YX_means_stdevs=YX_means_stdevs)
"DONE"

/Users/jeremymorton/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/jeremymorton/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Users/jeremymorton/.julia/v0.5/Conda/deps/usr/lib/python2.7/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/Use

In [ ]:
open(fout->write(fout, model), "GMR.txt", "w")
model.extractor.extractor.subset